In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

import sklearn.model_selection

%matplotlib inline

### Evaluation
Before doing anything else, we want to create a method to evaluate our models performance. This method should give back us evaluation metrics such as recall, f1, precision, and accuracy. Since we are doing binary prediction I will be using the code used for fraud detection but modifying it a little bit (including f1 score)

In [2]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1 = tp / (tp + 0.5*(fp + fn))
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print("{:<11} {:.3f}".format('F1:', f1))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### Data setup
To set up the sagemaker environment we are going to have to get our sessions, role, bucket, and data. We will also define the helper to delete an endpoint here

In [3]:
# sagemaker session, role, region
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket name
bucket = sagemaker_session.default_bucket()

In [4]:
X_test = pd.read_csv("data/X_test.csv", index_col=0)
Y_test = pd.read_csv("data/Y_test.csv", index_col=0)
X_train = pd.read_csv("data/X_train.csv", index_col=0)
Y_train = pd.read_csv("data/Y_train.csv", index_col=0)

In [5]:
X_train

,reward,difficulty,duration,mobile,social,web,bogo,discount,informational,age,income,F,M,O,days_since_member
20763,10,10,7,1,1,0,1,0,0,78,64000.0,0,1,0,407
35564,5,20,10,0,0,1,0,1,0,47,80000.0,1,0,0,1274
21422,2,10,7,1,0,1,0,1,0,77,83000.0,1,0,0,1569
44277,0,0,4,1,0,1,0,0,1,52,88000.0,0,1,0,1575
33385,2,10,10,1,1,1,0,1,0,63,70000.0,1,0,0,1354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324,5,5,7,1,0,1,1,0,0,53,56000.0,0,1,0,1765
37366,0,0,3,1,1,0,0,0,1,40,56000.0,0,1,0,1504
9707,2,10,10,1,1,1,0,1,0,50,84000.0,0,1,0,417
17014,10,10,5,1,1,1,1,0,0,23,36000.0,0,1,0,1039


In [6]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

### Linear Learner
We will first be using linear learner, we will try using linear learner raw first and then see how we do if we optimize it for recall. While we want a high f1 score, as that's our primary metric. It is probably more valuable for starbucks to get all potential customers and miss a few, than send promotions to those who would not use them. 

In [10]:
# import LinearLearner
from sagemaker import LinearLearner

# specify an output path
prefix = 'promotion_success'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner, no recall optimization
linear = LinearLearner(role=role,
                       instance_count=1, 
                       instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15)

In [22]:
# convert features/labels to numpy
X_train_np = X_train.astype('float32').values
Y_train_np = Y_train['Y'].astype('float32').values
X_test_np = X_test.astype('float32').values
Y_test_np = Y_test['Y'].astype('float32').values

# create RecordSet
formatted_train_data = linear.record_set(X_train_np, labels=Y_train_np)

In [34]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-03-13 04:29:56 Starting - Starting the training job...
2021-03-13 04:30:21 Starting - Launching requested ML instancesProfilerReport-1615609796: InProgress
......
2021-03-13 04:31:21 Starting - Preparing the instances for training.........
2021-03-13 04:32:54 Downloading - Downloading input data
2021-03-13 04:32:54 Training - Downloading the training image...
2021-03-13 04:33:23 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/13/2021 04:33:22 INFO 139684215854912] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_

[2021-03-13 04:33:24.188] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 461, "num_examples": 14, "num_bytes": 1420536}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.6707670569786659, "sum": 0.6707670569786659, "min": 0.6707670569786659}}, "EndTime": 1615610004.188365, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1615610004.188229}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.6701829880934496, "sum": 0.6701829880934496, "min": 0.6701829880934496}}, "EndTime": 1615610004.188481, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1615610004.188464}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.6717186795748197, "sum": 0.6


2021-03-13 04:33:43 Uploading - Uploading generated training model
2021-03-13 04:33:43 Completed - Training job completed
Training seconds: 58
Billable seconds: 58
CPU times: user 655 ms, sys: 45.8 ms, total: 701 ms
Wall time: 4min 14s


In [38]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------------!CPU times: user 356 ms, sys: 21.2 ms, total: 378 ms
Wall time: 10min 33s


In [39]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   X_test_np, 
                   Y_test_np, 
                   verbose=True) 

Metrics for simple, LinearLearner.

prediction (col)   0.0   1.0
actual (row)                
0.0               1817  1542
1.0               1162  2208

Recall:     0.655
Precision:  0.589
Accuracy:   0.598
F1:         0.620



In [40]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-03-13-04-36-11-963


In [41]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [42]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-03-13 04:49:07 Starting - Starting the training job...
2021-03-13 04:49:31 Starting - Launching requested ML instancesProfilerReport-1615610947: InProgress
......
2021-03-13 04:50:32 Starting - Preparing the instances for training.........
2021-03-13 04:52:05 Downloading - Downloading input data
2021-03-13 04:52:05 Training - Downloading the training image...
2021-03-13 04:52:36 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[03/13/2021 04:52:29 INFO 140250816034624] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'qu


2021-03-13 04:52:53 Completed - Training job completed
Training seconds: 46
Billable seconds: 46
CPU times: user 580 ms, sys: 40.5 ms, total: 621 ms
Wall time: 4min 12s


In [43]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------------!CPU times: user 419 ms, sys: 17.2 ms, total: 436 ms
Wall time: 12min 33s


In [44]:
print('Metrics for tuned (recall), LinearLearner.\n')

metrics = evaluate(recall_predictor, 
                   X_test_np, 
                   Y_test_np, 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)  0.0   1.0
actual (row)               
0.0               661  2698
1.0               333  3037

Recall:     0.901
Precision:  0.530
Accuracy:   0.550
F1:         0.667



In [45]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-03-13-04-53-33-198


### Set up XGBoost hyperparmeter tunning


In [7]:
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.33)

In [8]:
prefix = 'promotion_success/xgboost'
data_dir = 'data/sm'

X_test.to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([Y_val, X_val], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([Y_train, X_train], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = sagemaker_session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [14]:
from sagemaker.image_uris import retrieve
from sagemaker.xgboost.estimator import XGBoost

container = retrieve('xgboost', region, version='1.0-1')

xgb = sagemaker.estimator.Estimator(container, 
                                    role,     
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge', 
                                    output_path=output_path,
                                    sagemaker_session=sagemaker_session)

xgb.set_hyperparameters(max_depth=4,
                        eta=0.2,
                        min_child_weight=6,
                        objective='reg:logistic',
                        early_stopping_rounds=10,
                        num_round=200)

In [15]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost


xgb_hyperparameter_tuner = HyperparameterTuner(estimator = xgb,
                                               objective_metric_name = 'validation:f1', 
                                               objective_type = 'Maximize', 
                                               max_jobs = 20, 
                                               max_parallel_jobs = 3,
                                               hyperparameter_ranges = {
                                                    'max_depth': IntegerParameter(3, 8),
                                                    'eta'      : ContinuousParameter(0.05, 0.5),
                                                    'alpha'    : ContinuousParameter(0, 2),
                                                    'min_child_weight': IntegerParameter(1, 10),
                                               })

In [18]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

xgb_hyperparameter_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})
xgb_hyperparameter_tuner.wait()

......................................................................................................................................................................................................................................................................................................................................................................!
!


In [19]:
xgb_hyperparameter_tuner.best_training_job()

'sagemaker-xgboost-210313-1923-019-bc44ea89'

In [20]:
xgb_attached = sagemaker.estimator.Estimator.attach(xgb_hyperparameter_tuner.best_training_job())


2021-03-13 19:53:08 Starting - Preparing the instances for training
2021-03-13 19:53:08 Downloading - Downloading input data
2021-03-13 19:53:08 Training - Training image download completed. Training in progress.
2021-03-13 19:53:08 Uploading - Uploading generated training model
2021-03-13 19:53:08 Completed - Training job completed


In [24]:
xgb_predictor = xgb_attached.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-----------------------!

In [39]:
from sagemaker.serializers import CSVSerializer
xgb_predictor.serializer = CSVSerializer()

In [76]:
def evalution_metrics_xgb(test_labels, test_preds, verbose=True):
        
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1 = tp / (tp + 0.5*(fp + fn))
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print("{:<11} {:.3f}".format('F1:', f1))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}

In [77]:
test_preds = str(xgb_predictor.predict(X_test.to_csv(header=False, index=False))).split(',')
test_preds[0] = test_preds[0][2:]
test_preds[-1] = test_preds[-1][:-2]

In [78]:
test_preds = [round(float(x)) for x in test_preds]
_ = evalution_metrics_xgb(Y_test_np, np.asarray(test_preds))

prediction (col)     0     1
actual (row)                
0.0               1789  1570
1.0               1143  2227

Recall:     0.661
Precision:  0.587
Accuracy:   0.597
F1:         0.621



In [80]:
delete_endpoint(xgb_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted sagemaker-xgboost-2021-03-13-19-55-26-382
